In [143]:
import numpy as np
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt

from metpy.calc import dewpoint_from_relative_humidity
from metpy.units import units
from metpy.plots import SkewT

import sounding_utils
from xhistogram.xarray import histogram

from importlib import reload
from functools import partial
from joblib import dump

import sys
sys.path.append('../../') # lets us import ptype package from the subdir

#import ptype.

from dask.distributed import Client, LocalCluster
from dask_jobqueue import PBSCluster
import dask
import glob
from os.path import join

cluster = PBSCluster(account='NAML0001',
                     queue='cheyenne',
                     walltime='01:00:00',
                     memory="200 GB",
                     cores=24,
                     interface='ib0')

# Change your url to the dask dashboard so you can see it
dask.config.set({'distributed.dashboard.link':'https://jupyterhub.hpc.ucar.edu/stable/user/{USER}/proxy/{port}/status'})
                    
client = Client(cluster)
cluster.scale(jobs=10, memory="200GB")

cluster

In [3]:
model = 'rap'
a = glob.glob("/glade/scratch/dkimpara/ptype_case_studies/kentucky/{rap}/20220223/0000/*.nc")
files = a

In [52]:
%%time
ds = xr.open_mfdataset(files, parallel=False, engine='netcdf4', 
                       decode_cf=False, concat_dim='valid_time', combine='nested', 
                       chunks={'time':1, 'heightAboveGround': 21, 'y':337, 'x': 451, 'isobaricInhPa': 37})
ds.attrs['nwp'] = model

CPU times: user 2.3 s, sys: 578 ms, total: 2.88 s
Wall time: 2.92 s


In [53]:
ds

<xarray.Dataset>
Dimensions:                  (valid_time: 18, time: 1, heightAboveGround: 21,
                              y: 337, x: 451, isobaricInhPa: 37)
Coordinates:
  * heightAboveGround        (heightAboveGround) int64 0 250 500 ... 4750 5000
  * time                     (time) int64 1645574400
  * isobaricInhPa            (isobaricInhPa) float32 1e+03 975.0 ... 125.0 100.0
  * valid_time               (valid_time) float64 1.646e+09 ... 1.646e+09
Dimensions without coordinates: y, x
Data variables: (12/34)
    t_h                      (valid_time, time, heightAboveGround, y, x) float32 dask.array<chunksize=(1, 1, 21, 337, 451), meta=np.ndarray>
    dpt_h                    (valid_time, time, heightAboveGround, y, x) float32 dask.array<chunksize=(1, 1, 21, 337, 451), meta=np.ndarray>
    u_h                      (valid_time, time, heightAboveGround, y, x) float32 dask.array<chunksize=(1, 1, 21, 337, 451), meta=np.ndarray>
    v_h                      (valid_time, time, heightAboveGround, y, x) float32 dask.array<chunksize=(1, 1, 21, 337, 451), meta=np.ndarray>
    isobaricInhPa_h          (valid_time, time, heightAboveGround, y, x) float64 dask.array<chunksize=(1, 1, 21, 337, 451), meta=np.ndarray>
    step                     (valid_time) float64 1.0 13.0 10.0 ... 4.0 2.0 15.0
    ...                       ...
    ML_snow                  (valid_time, time, y, x) float32 dask.array<chunksize=(1, 1, 337, 451), meta=np.ndarray>
    ML_csnow                 (valid_time, time, y, x) uint8 dask.array<chunksize=(1, 1, 337, 451), meta=np.ndarray>
    ML_icep                  (valid_time, time, y, x) float32 dask.array<chunksize=(1, 1, 337, 451), meta=np.ndarray>
    ML_cicep                 (valid_time, time, y, x) uint8 dask.array<chunksize=(1, 1, 337, 451), meta=np.ndarray>
    ML_frzr                  (valid_time, time, y, x) float32 dask.array<chunksize=(1, 1, 337, 451), meta=np.ndarray>
    ML_cfrzr                 (valid_time, time, y, x) uint8 dask.array<chunksize=(1, 1, 337, 451), meta=np.ndarray>

In [155]:
def agg_stats(ds, save_dir='/glade/scratch/dkimpara/composite_calcs'):
    try:
        print(ds.attrs['nwp'])
    except: 
        raise ValueError('dataset must have nwp attr set')
        
    ds = sounding_utils.filter_latlon(ds)
    
    precip_mask = (
        (ds["crain"] == 1)
        | (ds["csnow"] == 1)
        | (ds["cicep"] == 1)
        | (ds["cfrzr"] == 1)
        )
    
    ds = ds.where(precip_mask)
    if 'wb_h' not in list(ds.keys()):
        ds = sounding_utils.wet_bulb_from_rel_humid(ds)
    
    ptypes = ['rain', 'snow', 'icep', 'frzr']
    prof_vars = ['t_h', 'dpt_h', 'wb_h']
    quantiles = np.arange(0,1.01,0.1)
    bins = np.arange(-40, 40, 0.5)
        
    #ds[prof_vars].persist() #edit this later
    total_obs = ds.t_h.count(dim=('x','y','time','valid_time')).max()

    datasets = []
    metadata = {}

    for ptype in ptypes:
        for model in ['ML_c', 'c']:
            predtype = model + ptype
            subset = ds[prof_vars].where(predtype == 1)
            
            ### num_obs per hr
            counts = subset.t_h.count(dim=('x','y'))
            obs_per_hr = counts.mean(dim=('time', 'valid_time')).max()
            
            # num_obs of predtype==1
            num_obs = subset.t_h.count(dim=('x','y','time','valid_time')).max()
            
            # num_obs w frac abv zero
            for var in prof_vars:
                metadata[var+'frac_abv_zero'] = (
                    sounding_utils.frac_abv_zero(subset, var, num_obs)
                )
            metadata[f'{predtype}_obs_per_hr'] = obs_per_hr
            
            # means and quantiles
            mean = subset.mean(dim=('valid_time', 'time', 'x', 'y')) #dataset types
            mean = mean.rename({var: f'{var}_mean' for var in prof_vars})

            qs = subset.chunk(dict(time=-1)).quantile(quantiles, dim=('valid_time','time', 'x', 'y'))
            qs = qs.rename({var: f'{var}_qs' for var in prof_vars})
            
            #### densities ####
            densities = {}
            for var in prof_vars:
                densities[var] = (
                    (histogram(subset[var], bins=bins, dim=['valid_time', 'time', 'x', 'y'])
                     .rename({f'{var}_bin': 'bin'})
                    )
                )
            densities = xr.Dataset(densities) #arrays already named histograms
            ############ concat and finish up #############
            res = xr.concat((mean, qs, densities),
                            dim='heightAboveGround')
            res = res.expand_dims({'predtype': [predtype]})
            datasets.append(res)
    ################# by confidence level ###################
    for ptype in ptypes:
        predtype = f'ML_{ptype}'
        subset = ds[prof_vars].where(ds[predtype] >= 0.7)
        
        mean = subset.mean(dim=['valid_time', 'time', 'x', 'y'])
        mean = mean.rename({var: f'{var}>0.7_mean' for var in prof_vars})
        qs = subset.chunk(chunks=dict(valid_time=-1)).quantile(quantiles, dim=('valid_time','time', 'x', 'y'))
        qs = qs.rename({var: f'{var}>0.7_qs' for var in prof_vars})
        
        datasets.append(xr.concat((mean,qs), dim='heightAboveGround')
                        .expand_dims({'predtype': [predtype]}))
        
        predtype = f'c{ptype}'
        subset = subset.where(ds[predtype] == 1) #confident ML and nwp predicts
        mean = subset.mean(dim=['valid_time', 'time', 'x', 'y'])
        mean = mean.rename({var: f'{var}>0.7_mean' for var in prof_vars})
        qs = subset.chunk(chunks=dict(valid_time=-1)).quantile(quantiles, dim=('valid_time','time', 'x', 'y'))
        qs = qs.rename({var: f'{var}>0.7_qs' for var in prof_vars})
        
        datasets.append(xr.concat((mean,qs), dim='heightAboveGround')
                .expand_dims({'predtype': [predtype]}))
        
    ####################################
    result = xr.concat(datasets, dim='heightAboveGround', coords='minimal')
    result = result.rename({f'{var}': f'{var}_hist' for var in prof_vars})
    result.to_netcdf(path=join(save_dir, ds.attrs['nwp']))
    dump(metadata, join(save_dir,f'{ds.attrs}_metadata'))
    
    return result
    

In [ ]:
'''
client.shutdown()
import subprocess
subprocess.run("qdel $PBS_JOBID", shell=True, capture_output=True, encoding='utf-8')